### IMPORT DEPEDENCIES

In [1]:
from pwp.utils import Preprocessing,Visualization,\
     IdealFunction, SqliteSqlalchemy
from bokeh.plotting import figure, output_notebook, show
import pandas as pd
import numpy as np

### Import train.csv, test.csv, and ideal.csv files

In [2]:
# import train.csv, test.csv, and ideal.csv files
# Create an object of the class Preprocessing 
prep = Preprocessing() 
train = prep.import_dataset('/home/kgyasi/Documents/iubh-pwp/train .csv')  # reads in train train.csv file 
test = prep.import_dataset('/home/kgyasi/Documents/iubh-pwp/test.csv')  # reads in the test.csv file 
ideal = prep.import_dataset('/home/kgyasi/Documents/iubh-pwp/ideal.csv')  # reads in the ideal.csv file 

In [3]:
# preview train
train.head(3)

,x,y1,y2,y3,y4
0,-20.0,4.947051,9.139399,-0.025432,4.760395
1,-19.9,5.157783,9.082572,-0.103177,4.512951
2,-19.8,5.923819,9.407426,0.425233,4.669478


In [4]:
# preview test
test.head(3)

,x,y
0,6.6,6.777071
1,6.7,2.097181
2,18.6,4.626871


In [5]:
# preview ideal
print(ideal.head(2))
print(ideal.shape)


      x        y1        y2        y3        y4        y5        y6        y7  \
0 -20.0 -0.912945  0.408082  9.087055  5.408082 -9.087055  0.912945 -0.839071   
1 -19.9 -0.867644  0.497186  9.132356  5.497186 -9.132356  0.867644 -0.865213   

         y8        y9  ...        y41       y42       y43       y44  \
0 -0.850919  0.816164  ... -40.456474  40.20404  2.995732 -0.008333   
1  0.168518  0.994372  ... -40.233820  40.04859  2.990720 -0.008340   

         y45       y46       y47       y48       y49       y50  
0  12.995732  5.298317 -5.298317 -0.186278  0.912945  0.396850  
1  12.990720  5.293305 -5.293305 -0.215690  0.867644  0.476954  

[2 rows x 51 columns]
(400, 51)


### Get dependent and independent variables

In [6]:
# Split dataframe into dependent(X) and independent(Y) values 
X_train, Y_train = Preprocessing.values(train) 
X_test, Y_test = Preprocessing.values(test) 
X_ideal, Y_ideal = Preprocessing.values(ideal) 

### Visualise the Training data and test data 

In [7]:
visual = Visualization()

In [8]:
# Visualise train data
number = 1
for i in train.iloc[:, 1:]:
    visual.plot_bokeh("x", i, f"Training Data {number}", train)
    number +=1 

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [9]:
# Visualise test data
number = 1
for i in test.iloc[:, 1:]:
    visual.plot_bokeh("x", i, f"Test Points {number}", test)
    number +=1 

Loading BokehJS ...

### IDEAL FUNCTIONS FOR TRAIN DATA

In [10]:
# Extract the Y values from the ideal data 
Y_ideal_col = Y_ideal.iloc[:].values 
# print(Y_ideal_col 

# Determine the ideal function for each training data 

id_fun = IdealFunction()  # Object of the class ideal function 
ideal_fn = list()
train_col = list()
number = 1
for i in Y_train.iloc[:]:
    train_col.append(i)
    lse = id_fun.locate_ideal_fun(Y_train[i], Y_ideal_col, Y_ideal) 
    ideal_fn.append(lse)
    print(f'The ideal function of training data {number} is the ideal data column :{lse}') 
    # Visualize the ideal functions 
    visual.plot_bokeh("x", lse, f"Ideal function for train data {number}", ideal) 
    number +=1

The ideal function of training data 1 is the ideal data column :y4


Loading BokehJS ...

The ideal function of training data 2 is the ideal data column :y3


Loading BokehJS ...

The ideal function of training data 3 is the ideal data column :y44


Loading BokehJS ...

The ideal function of training data 4 is the ideal data column :y32


Loading BokehJS ...

### Determine the maximum deviation between each training data and its corresponding ideal function 

In [11]:
max_deviation = list()
for i in range(0, len(ideal_fn)):
    ideal_id = ideal_fn[i]
    train_id = train_col[i]
    max_dev = IdealFunction.max_deviation(Y_train[train_id], Y_ideal[ideal_id])
    max_deviation.append(max_dev)
    print(
        f"The maximum deviation between the train data {i+1} and it's ideal function is:{max_dev:.5f}")

The maximum deviation between the train data 1 and it's ideal function is:0.49743
The maximum deviation between the train data 2 and it's ideal function is:0.49632
The maximum deviation between the train data 3 and it's ideal function is:0.49999
The maximum deviation between the train data 4 and it's ideal function is:0.49621


### Determine the 100 ideal points from the 400 ideal function coordinates for the four  ideal functions and calculate the deviation from the test points 

 Ceate an ideal function table

In [12]:
ideal_table = (ideal['x'], ideal['y4'], ideal['y3'], ideal['y44'],  ideal['y32']) 
ideal_table = pd.DataFrame(ideal_table).transpose() 
ideal_table.head(3)

,x,y4,y3,y44,y32
0,-20.0,5.408082,9.087055,-0.008333,4.472136
1,-19.9,5.497186,9.132356,-0.008340,4.460942
2,-19.8,5.581322,9.186326,-0.008347,4.449719


In [13]:
dev, id_table = id_fun.deviations(X_test, X_ideal, ideal_table, Y_test.values) 
#print(dev) 
#id_table[:4]

In [14]:
# create a table of the test points and the ideal coordinates 
id_table = np.array(id_table).transpose() 
id_table = {'x': id_table[0].flatten(), 
            'y4': id_table[1].flatten(), 
            'y3': id_table[2].flatten(), 
            'y44': id_table[3].flatten(), 
            'y32': id_table[4].flatten() 
            } 

#id_table

### Compare the deviation to the conditional statement

In [15]:
label = 1
dif = dict()
for i in range(0, len(max_deviation)):

    d1 = id_fun.dev_mapper(dev[i], max_deviation[i]) 
    dif[label]=d1
    label +=1
# # Create a table of the deviations outcome and output the minimum of each row 
diff = pd.DataFrame(dif) 
# The function with the least deviation will be the most ideal for a test point 

diff['min'] = diff.min(axis=1) 
dev_min = np.array(diff['min'])  # convert to NumPy array 

### Map the deviations to the ideal functions 

In [16]:
func = {}
label = 1
for i in range(1, len(ideal_fn)+1):
    f1 = id_fun.func_map(dev_min, dif[i], i)
    func[label] = f1
    label +=1

func = pd.DataFrame(func)
# get the mean
func = func.mean(axis=1)
#print(func)

### Create a table of the test-data, with mapping and y-deviation 

In [17]:
df = test 
df['delta'] = diff['min']
df['No. of Ideal Function'] = func
df.style.set_table_attributes("style='display:inline'").set_caption('Table of test-data, with mapping and y-deviations') 

,x,y,delta,No. of Ideal Function
0,6.600000,6.777071,nan,nan
1,6.700000,2.097181,0.491255,4.000000
2,18.600000,4.626871,0.314099,4.000000
3,3.400000,8.583755,nan,nan
4,8.800000,0.401559,0.412524,3.000000
5,-12.700000,7.358367,nan,nan
6,-1.300000,4.631897,0.635602,1.000000
7,10.000000,3.112941,0.049337,4.000000
8,-9.900000,4.367376,0.256567,1.000000
9,5.300000,10.026350,nan,nan


### visualize the test points, the ideal functions

In [18]:
# define tooltips for bokeh
TOOLTIPS = [
("index", "$index"),
("(x,y)", "($x, $y)")]
# plot
p = figure(title='Ideal functions and test points', tooltips=TOOLTIPS)
p.circle(ideal['x'], ideal['y4'], color='red', legend_label='Ideal Funtion 1')
p.circle(ideal['x'], ideal['y3'], color='green', legend_label='Ideal Funtion 2')
p.circle(ideal['x'], ideal['y44'], color='indigo', legend_label='Ideal Funtion 3')
p.circle(ideal['x'], ideal['y32'], color='blue', legend_label='Ideal Funtion 4')
p.circle(test['x'], test['y'], color='black', legend_label='Test points')
p.legend.location = "top_right"
output_notebook()
show(p)

Loading BokehJS ...

In [19]:
p = figure(title='Ideal functions and test points under apropriate chosen deviation',
tooltips=TOOLTIPS)
p.circle(id_table['x'], id_table['y4'], color='red', legend_label='Ideal function 1')
p.circle(id_table['x'], id_table['y3'], color='green', legend_label='Ideal function 2')
p.circle(id_table['x'], id_table['y44'], color='indigo', legend_label='Ideal function 3')
p.circle(id_table['x'], id_table['y32'], color='blue', legend_label='Ideal function 4')
p.circle(test['x'], test['y'], color='black', legend_label='Test points')
p.legend.location = "top_right"
output_notebook()
show(p)

Loading BokehJS ...

### Save final table to the current directory

In [20]:
df.to_csv(r'./final_table.csv', index=False, header=True)

### Importing CSV files into SQLite database assignment.db

In [21]:
sqlite_sqlalchemy = SqliteSqlalchemy() # object of the class SqliteSqlalchemy
# Import the train.csv
sqlite_sqlalchemy.import_csv_table("train .csv", "Table 1: The training data's database table",
"assignment")
# Import the idealfunction table
sqlite_sqlalchemy.import_csv_table("ideal.csv", "Table 2: The ideal functions’ database table",
"assignment")
# Import the database table of the test-data, with mapping and y-deviation
sqlite_sqlalchemy.import_csv_table("final_table.csv", "Table 3:The database table of the test-data,with mapping and y-deviation", "assignment")

2023-01-21 19:05:02,226 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Table 1: The training data's database table")
2023-01-21 19:05:02,227 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-21 19:05:02,238 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-21 19:05:02,249 INFO sqlalchemy.engine.Engine INSERT INTO "Table 1: The training data's database table" ("index", x, y1, y2, y3, y4) VALUES (?, ?, ?, ?, ?, ?)
2023-01-21 19:05:02,251 INFO sqlalchemy.engine.Engine [generated in 0.00791s] ((0, -20.0, 4.9470515, 9.139399, -0.025432454, 4.7603946), (1, -19.9, 5.1577826, 9.082572, -0.103176646, 4.512951), (2, -19.8, 5.923819, 9.407426, 0.42523292, 4.6694784), (3, -19.7, 6.0915623, 8.953618, 0.13769506, 4.414474), (4, -19.6, 6.2020335, 9.593519, -0.11682482, 4.2050943), (5, -19.5, 5.9927864, 9.417254, -0.096975856, 4.724585), (6, -19.4, 5.8405395, 9.537546, 0.32761803, 4.2226596), (7, -19.3, 5.668819, 9.415276, -0.43552196, 4.448568)  ... displaying 10 of 400 total bound param